> ### Note on Labs and Assigments:
>
> 🔧 Look for the **wrench emoji** 🔧 — it highlights where you're expected to take action!
>
> These sections are graded and are not optional.
>

# IS 4487 Lab 6: Data Cleaning

## Outline

- Load and inspect a new dataset (Megatelco)
- Fix column names and data types
- Handle missing values
- Remove duplicate rows
- Review and remove outliers
- Reflect on data quality

In this lab, we’ll clean the data to get it ready for transformations and analysis.

We will continue working with this dataset in **Lab 7**, where we will create new features and apply transformations.

<a href="https://colab.research.google.com/github/Stan-Pugsley/is_4487_base/blob/main/Labs/lab_06_data_cleaning.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Megatelco Data Dictionary

 DEMOGRAPHIC VARIABLES:
 - College - has the customer attended some college (one, zero)
 - Income - annual income of customer
 - House - estimated price of the customer's home (if applicable)

 USAGE VARIABLES:
 - Data Overage Mb - Average number of megabytes that the customer used in excess of the plan limit (over last 12 months)
 - Data Leftover Mb - Average number of megabytes that the customer use was below the plan limit (over last 12 months)
 - Data Mb Used - Average number of megabytes used per month (over last 12 months)
 - Text Message Count - Average number of texts per month (over last 12 months)
 - Over 15 Minute Calls Per Month - Average number of calls over 15 minutes in duration per month (over last 12 months)
 - Average Call Duration- Average call duration (over last 12 months)

PHONE VARIABLES:
 - Operating System - Current operating system of phone
 - Handset Price - Retail price of the phone used by the customer

ATTITUDINAL VARIABLES:
 - Reported Satisfaction - Survey response to "How satisfied are you with your current phone plan?" (high, med, low)
 - Reported Usage Level - Survey response to "How much do your use your phone?" (high, med, low)
 - Considering Change of Plan - Survey response to "Are you currently planning to change companies when your contract expires?" (high, med, low)

OTHER VARIABLES
 - Leave - Did this customer churn with the last contract expiration? (LEAVE, STAY)
 - ID - Customer identifier

In [1]:
import pandas as pd

url = "https://raw.githubusercontent.com/Stan-Pugsley/is_4487_base/812e9f15c357a5657a2795631fcaa9d9363cb417/DataSets/megatelco_leave_survey_data_cleaning_v2.csv"
df = pd.read_csv(url)

df.head()

,college,income,data_overage_mb,data_leftover_mb,data_mb_used,text_message_count,house,handset_price,over_15mins_calls_per_month,average_call_duration,reported_satisfaction,reported_usage_level,considering_change_of_plan,leave,id,operating_system
0,one,403137.0,70,0.0,6605.0,199,841317,653.0,5.0,8.0,low,low,yes,LEAVE,8183,Android
1,zero,129700.0,67,16.0,6028.0,134,476664,1193.0,5.0,5.0,low,low,yes,LEAVE,12501,IOS
2,zero,69741.0,60,0.0,1482.0,176,810225,1037.0,3.0,8.0,low,low,yes,STAY,7425,IOS
3,one,377572.0,0,22.0,3005.0,184,826967,1161.0,0.0,5.0,low,low,no,LEAVE,13488,IOS
4,zero,382080.0,0,0.0,1794.0,74,951896,1023.0,0.0,14.0,low,low,yes,STAY,11389,IOS


In [2]:
# create a copy of your dataset for use in part 4
copied_df = df.copy(deep=True)

## Part 1: Review Column Names and Structure

Before cleaning, check the structure of the dataset:

- Are column names consistent (lowercase, no spaces)?
- Are there any typos or redundant labels?
- Do the rows and columns appear aligned?

Why this matters:
Inconsistent or messy column names can break code and make analysis harder to follow.




In [3]:
# Standardize column names: lowercase, no spaces
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

# Get column info and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15016 entries, 0 to 15015
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   college                      15016 non-null  object 
 1   income                       15006 non-null  float64
 2   data_overage_mb              15016 non-null  int64  
 3   data_leftover_mb             14916 non-null  float64
 4   data_mb_used                 14916 non-null  float64
 5   text_message_count           15016 non-null  int64  
 6   house                        15016 non-null  int64  
 7   handset_price                14916 non-null  float64
 8   over_15mins_calls_per_month  15013 non-null  float64
 9   average_call_duration        14916 non-null  float64
 10  reported_satisfaction        15016 non-null  object 
 11  reported_usage_level         15016 non-null  object 
 12  considering_change_of_plan   14201 non-null  object 
 13  leave           

## Part 2: Convert Data Types

Before analysis, make sure each column is stored in the correct format. This helps avoid calculation errors, makes plotting smoother, and ensures models interpret the data correctly.

Think about:
- Are numbers accidentally stored as strings?
- Should repeated text values be converted to categories?
- Are "yes"/"no" columns better represented as binary (0/1) or categorical types?

Fixing data types now saves time and avoids issues later in your workflow.




In [4]:
# Check original data types
print("Original dtypes:\n", df.dtypes)

# Convert yes/no to binary categories
df['considering_change_of_plan'] = df['considering_change_of_plan'].map({'yes': 1, 'no': 0}).astype('category')

# Convert categorical text columns
df['college'] = df['college'].astype('category')
df['house'] = df['house'].astype('category')
df['reported_satisfaction'] = df['reported_satisfaction'].astype('category')
df['operating_system'] = df['operating_system'].astype('category')

# Convert numeric-looking columns from object to float
df['income'] = pd.to_numeric(df['income'], errors='coerce')
df['data_overage_mb'] = pd.to_numeric(df['data_overage_mb'], errors='coerce')
df['data_leftover_mb'] = pd.to_numeric(df['data_leftover_mb'], errors='coerce')
df['data_mb_used'] = pd.to_numeric(df['data_mb_used'], errors='coerce')
df['text_message_count'] = pd.to_numeric(df['text_message_count'], errors='coerce')
df['over_15mins_calls_per_month'] = pd.to_numeric(df['over_15mins_calls_per_month'], errors='coerce')
df['average_call_duration'] = pd.to_numeric(df['average_call_duration'], errors='coerce')

# Check updated data types
print("\nUpdated dtypes:\n", df.dtypes)


Original dtypes:
 college                         object
income                         float64
data_overage_mb                  int64
data_leftover_mb               float64
data_mb_used                   float64
text_message_count               int64
house                            int64
handset_price                  float64
over_15mins_calls_per_month    float64
average_call_duration          float64
reported_satisfaction           object
reported_usage_level            object
considering_change_of_plan      object
leave                           object
id                               int64
operating_system                object
dtype: object

Updated dtypes:
 college                        category
income                          float64
data_overage_mb                   int64
data_leftover_mb                float64
data_mb_used                    float64
text_message_count                int64
house                          category
handset_price                   float64
over_1

### 🔧 Try It Yourself – Part 2

1. Convert the `leave` column from "yes"/"no" to binary (`1`/`0`) and make it a **category**
2. Convert `reported_usage_level` to a **categorical** type
3. Convert `house` to an **integer** type
3. Use `.info()` to confirm the changes


In [8]:
# add code here 🔧
# Convert 'leave' from 'yes'/'no' to binary 1/0 and make it a category
df['leave'] = df['leave'].map({'yes': 1, 'no': 0}).astype('category')

# Convert 'reported_usage_level' to categorical
df['reported_usage_level'] = df['reported_usage_level'].astype('category')

# Convert 'house' to numeric, fill missing with 0, then to integer
df['house'] = pd.to_numeric(df['house'], errors='coerce').fillna(0).astype(int)

# Confirm changes
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15016 entries, 0 to 15015
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   college                      15016 non-null  category
 1   income                       15006 non-null  float64 
 2   data_overage_mb              15016 non-null  int64   
 3   data_leftover_mb             14916 non-null  float64 
 4   data_mb_used                 14916 non-null  float64 
 5   text_message_count           15016 non-null  int64   
 6   house                        15016 non-null  int64   
 7   handset_price                14916 non-null  float64 
 8   over_15mins_calls_per_month  15013 non-null  float64 
 9   average_call_duration        14916 non-null  float64 
 10  reported_satisfaction        15016 non-null  category
 11  reported_usage_level         15016 non-null  category
 12  considering_change_of_plan   0 non-null      category
 13  l

## Part 3: Handle Missing Values

Missing data can break charts, skew stats, and disrupt models — so it needs to be handled carefully.

### Think about:
- Are the missing values random or patterned?
- Can we drop rows, or do we need to fill them?
- Should we use mean, median, or something else?

### Guidelines:
- Drop rows only if few are missing and the column is essential.
- Use **median** for numeric columns with outliers.
- Use **0** if missing logically means "none" (e.g., no leftover data).
- Use **mode** for categorical values.

Cleaning missing values early avoids bigger problems later.

-----


**Note on `.loc` and Warnings** - When assigning values to a DataFrame, especially after filtering or copying, it's best to use `.loc` to avoid **`SettingWithCopyWarning`**. This ensures that you're updating the original data and not a temporary view of it.


In [9]:
# View missing value counts
print("Missing values per column:\n", df.isnull().sum())

# Fill 'handset_price' with median
df['handset_price'] = df['handset_price'].fillna(df['handset_price'].median())

# Drop rows with missing 'income' (if very few)
df = df.dropna(subset=['income']).copy()

# Fill missing 'data_leftover_mb' with 0 if it logically means no leftover data
df.loc[:, 'data_leftover_mb'] = df['data_leftover_mb'].fillna(0)

# Fill 'average_call_duration' with median if necessary
df.loc[:, 'average_call_duration'] = df['average_call_duration'].fillna(df['average_call_duration'].median())

# Fill 'data_mb_used' with median
df.loc[:, 'data_mb_used'] = df['data_mb_used'].fillna(df['data_mb_used'].median())

# Confirm updated missing values
print("\nMissing values after handling:\n", df.isnull().sum())


Missing values per column:
 college                            0
income                            10
data_overage_mb                    0
data_leftover_mb                 100
data_mb_used                     100
text_message_count                 0
house                              0
handset_price                    100
over_15mins_calls_per_month        3
average_call_duration            100
reported_satisfaction              0
reported_usage_level               0
considering_change_of_plan     15016
leave                          15016
id                                 0
operating_system                   0
dtype: int64

Missing values after handling:
 college                            0
income                             0
data_overage_mb                    0
data_leftover_mb                   0
data_mb_used                       0
text_message_count                 0
house                              0
handset_price                      0
over_15mins_calls_per_month        3
a

### 🔧 Try It Yourself – Part 3


There are still some missing values in:

- `over_15mins_calls_per_month`
- `considering_change_of_plan`

Decide how to handle them based on what makes the most sense:

- Should you fill them with 0, the median, or something else?
- For categories, would a placeholder like "unknown" or the most common value work?
- Or is it better to drop those rows?

1. Write and execute code to handle the missing values in the remaining two columns.
2. Use `df.isnull().sum()` to confirm all missing values are handled.



In [12]:
# Add code here 🔧
# Handle missing values in 'over_15mins_calls_per_month' using median
median_calls = df['over_15mins_calls_per_month'].median()
df['over_15mins_calls_per_month'] = df['over_15mins_calls_per_month'].fillna(median_calls)

# Handle missing values in 'considering_change_of_plan'
# Get the mode, fallback to 'unknown'
if not df['considering_change_of_plan'].mode().empty:
    mode_considering = df['considering_change_of_plan'].mode()[0]
else:
    mode_considering = 'unknown'

# Ensure mode_considering is in the category list before filling
if mode_considering not in df['considering_change_of_plan'].cat.categories:
    df['considering_change_of_plan'] = df['considering_change_of_plan'].cat.add_categories([mode_considering])

# Fill missing values with mode
df['considering_change_of_plan'] = df['considering_change_of_plan'].fillna(mode_considering)

# Confirm all missing values handled
print(df.isnull().sum())




college                            0
income                             0
data_overage_mb                    0
data_leftover_mb                   0
data_mb_used                       0
text_message_count                 0
house                              0
handset_price                      0
over_15mins_calls_per_month        0
average_call_duration              0
reported_satisfaction              0
reported_usage_level               0
considering_change_of_plan         0
leave                          15006
id                                 0
operating_system                   0
dtype: int64


## Part 4: Remove Duplicate Rows

Sometimes the same row appears more than once due to data entry or processing mistakes. It's important to check for and remove these duplicates.

Think about:
- Are there rows that are exactly the same?
- If duplicates exist, should you keep the first one, the last one, or none?

Why this matters:
Duplicate rows can inflate totals, distort statistics, and lead to inaccurate conclusions.


In [13]:
# Check for exact duplicates
print(f"Number of duplicate rows: {df.duplicated().sum()}")

# Remove them, keeping the first occurrence
df = df.drop_duplicates()

# Confirm result
print(f"Remaining rows after removing duplicates: {len(df)}")

Number of duplicate rows: 17
Remaining rows after removing duplicates: 14989


### 🔧 Try It Yourself – Part 4

1. Use `copied_df.duplicated().sum()` to count how many duplicates are in your dataset.
2. Try using `copied_df.drop_duplicates(keep='last')` instead — what changes?
3. Explore whether duplicate rows share the same ID or just values across all columns and comment on your observation.


In [14]:
# 🔧 Add code here:
# Step 1: Make a copy to preserve original
copied_df = df.copy()

# Step 2: Count number of full row duplicates
duplicate_count = copied_df.duplicated().sum()
print("Number of duplicate rows:", duplicate_count)

# Step 3: Drop duplicates using 'keep=last' and check resulting shape
deduped_df = copied_df.drop_duplicates(keep='last')
print("Shape after dropping duplicates (keep last):", deduped_df.shape)

# Step 4: Compare row count before and after
print("Original shape:", copied_df.shape)
print("Rows removed:", copied_df.shape[0] - deduped_df.shape[0])

# Step 5: Explore if duplicate rows share same ID
# Find actual duplicate rows
duplicate_rows = copied_df[copied_df.duplicated(keep=False)]

# Check if their IDs are the same
id_unique_in_dupes = duplicate_rows['id'].nunique()
total_dupe_rows = duplicate_rows.shape[0]

print("Number of duplicate rows (total):", total_dupe_rows)
print("Unique IDs in duplicate rows:", id_unique_in_dupes)


Number of duplicate rows: 0
Shape after dropping duplicates (keep last): (14989, 16)
Original shape: (14989, 16)
Rows removed: 0
Number of duplicate rows (total): 0
Unique IDs in duplicate rows: 0


🔧 Add comment here: There are no duplicate rows in the dataset. This indicates that each customer record is distinct, including all of the associated variables.

## Part 5: Identify and Remove Obvious Outliers

Outliers are values that fall far outside the normal range. They can come from data entry mistakes or rare cases.

- Use summary statistics or visual tools (like boxplots) to find them.
- Look for clearly unrealistic values — e.g., negative prices or extremely high data usage.
- Decide how to handle them:
  - Remove if they’re errors.
  - Keep if they’re valid but rare — or cap them if needed.

Outliers can distort averages, stretch visualizations, and mislead models, so it’s important to address them carefully.



In [15]:
# Remove negative or nonsensical values using business rules

# Example: remove rows where 'handset_price' is negative
df = df[df['handset_price'] >= 0]

# Example: remove rows with unusually long call durations
df = df[df['average_call_duration'] < 1000]

# Example: remove rows with extremely high text message counts
df = df[df['text_message_count'] < 1000]

# View shape after outlier filtering
print("Shape after removing obvious outliers:", df.shape)


Shape after removing obvious outliers: (14986, 16)


### 🔧 Try It Yourself – Part 5

1. Use `df.describe()` to look for columns with extreme minimum or maximum values.
2. Set a threshold for what you think is "too high" or "too low" for:
  - `data_mb_used`
  - `over_15mins_calls_per_month`
  - `income`
3. Remove those outliers using boolean filtering like `df = df[df['column'] < threshold]`

In [17]:
# 🔧 add code here:
# Step 1: Summary statistics to inspect potential outliers
print(df[['data_mb_used', 'over_15mins_calls_per_month', 'income']].describe())

# Step 2: Set thresholds for unrealistic high values
data_mb_threshold = 50000         # More than 50 GB/month seems unrealistic
calls_threshold = 100             # Over 100 long calls/month is excessive
income_threshold = 1_000_000      # Over $1M likely a data entry issue

# Step 3: Remove high-end outliers
df = df[df['data_mb_used'] < data_mb_threshold]
df = df[df['over_15mins_calls_per_month'] < calls_threshold]
df = df[df['income'] < income_threshold]

# Step 4: Remove clearly invalid negative income values
df = df[df['income'] >= 0]

# Step 5: Confirm that outliers are removed
print("\nAfter outlier removal:")
print(df[['data_mb_used', 'over_15mins_calls_per_month', 'income']].describe())
print("New shape:", df.shape)



       data_mb_used  over_15mins_calls_per_month         income
count  14986.000000                 14986.000000   14986.000000
mean    4201.313760                    10.569331  241977.965768
std     2196.751708                     8.401171  109612.549926
min      400.000000                     0.000000  -65000.000000
25%     2304.000000                     3.000000  147808.250000
50%     4221.000000                     9.000000  241653.000000
75%     6063.000000                    17.000000  336442.000000
max     8000.000000                    35.000000  432000.000000

After outlier removal:
       data_mb_used  over_15mins_calls_per_month        income
count  14984.000000                 14984.000000   14984.00000
mean    4201.413374                    10.570675  242016.73752
std     2196.609956                     8.400924  109568.30733
min      400.000000                     0.000000   52491.00000
25%     2304.000000                     3.000000  147825.50000
50%     4221.000000   

## Part 6: Handle Outliers Using Quantiles

Instead of removing outliers, we can limit their impact by capping extreme values — a method known as **Winsorizing**.

### How to Do It:
- Use `.quantile()` to identify the 1st and 99th percentiles (or other thresholds).
- Use `.clip()` to cap values within that range.

This keeps your dataset intact while reducing the influence of extreme values on your analysis or model.



In [18]:
# Calculate 1st and 99th percentiles for income
income_min, income_max = df['income'].quantile([0.01, 0.99])

# Use .loc to avoid SettingWithCopyWarning and ensure assignment modifies the original DataFrame
df.loc[:, 'income'] = df['income'].clip(lower=income_min, upper=income_max)

# Clip 'data_mb_used' to within 1st and 99th percentiles
usage_min, usage_max = df['data_mb_used'].quantile([0.01, 0.99])
df.loc[:, 'data_mb_used'] = df['data_mb_used'].clip(lower=usage_min, upper=usage_max)

# Clip 'average_call_duration' to reduce the effect of extreme outliers
call_min, call_max = df['average_call_duration'].quantile([0.01, 0.99])
df.loc[:, 'average_call_duration'] = df['average_call_duration'].clip(lower=call_min, upper=call_max)



### 🔧 Try It Yourself – Part 6

1. Use `.quantile([0.01, 0.99])` to find the range for:
  - `text_message_count`
  - `over_15mins_calls_per_month`
2. Apply `.clip(lower=..., upper=...)` to reduce the impact of those outliers
3. Compare the `.describe()` output before and after clipping and comment on what you observe


In [19]:
# 🔧 Add code here
# Step 1: Store original describe stats for comparison
before_clip = df[['text_message_count', 'over_15mins_calls_per_month']].describe()

# Step 2: Calculate 1st and 99th percentiles for each column
text_msg_bounds = df['text_message_count'].quantile([0.01, 0.99])
calls_bounds = df['over_15mins_calls_per_month'].quantile([0.01, 0.99])

# Step 3: Apply clipping (Winsorization)
df['text_message_count'] = df['text_message_count'].clip(lower=text_msg_bounds[0.01], upper=text_msg_bounds[0.99])
df['over_15mins_calls_per_month'] = df['over_15mins_calls_per_month'].clip(lower=calls_bounds[0.01], upper=calls_bounds[0.99])

# Step 4: Store describe stats after clipping
after_clip = df[['text_message_count', 'over_15mins_calls_per_month']].describe()

# Step 5: Display both for comparison
print("Before Clipping:\n", before_clip)
print("\nAfter Clipping:\n", after_clip)


Before Clipping:
        text_message_count  over_15mins_calls_per_month
count        14984.000000                 14984.000000
mean           135.624132                    10.570675
std             48.837639                     8.400924
min             52.000000                     0.000000
25%             93.000000                     3.000000
50%            135.000000                     9.000000
75%            178.000000                    17.000000
max            220.000000                    35.000000

After Clipping:
        text_message_count  over_15mins_calls_per_month
count        14984.000000                 14984.000000
mean           135.623465                    10.559597
std             48.814121                     8.370176
min             53.000000                     0.000000
25%             93.000000                     3.000000
50%            135.000000                     9.000000
75%            178.000000                    17.000000
max            219.000000    

🔧 Add comment here: After clipping, the maximum values for both textmessagecount and over15minscallspermonth were reduced slightly. This brought them in line with the 99th percentile and lowered the influence of extreme outliers. This adjustment had minimal impact on the mean and the stdev, helping to preserve the overall distribution while reducing potential skew from rare high values.

## 🔧 Part 7: Reflection (100 words or less per question)

1. Which step fixed the most issues in the dataset?
2. What surprised you about the structure or values?
3. Do you feel this data is now ready for transformation in Lab 7?


🔧 **Add comment here:**

1. Handling missing values and correcting missing data types was a great way to resolve a lot of issues. Several solumns had nulls or incorrect formats, this like categorical data stored as strings or numerics stored with invalid values, which could have broken later analysis. Fillin gmissing values with the medians, modes, or realistic defaults made the data much more complete and much more reliable.

2. It was surprising to see a negative income value and how few duplicate rows existed despite the dataset's size. Also, some values like very high mobile data usage or long call counts seemed innacurate and not realistic. This highlights how outliers can distort summary statistics without being obvious at first glance.

3. Yes, the dataset is now clean, consistent, and free of missing or extreme outlier issues. WIth validated datatypes, fixed formating, and capped extreme values, its well prepared for anyone who wants to look at it and it is ready to be interpreted.
---




## Export Your Notebook to Submit in Canvas
- Use the instructions from Lab 1

In [20]:
!jupyter nbconvert --to html "lab_06_WoodMason.ipynb"

[NbConvertApp] Converting notebook lab_06_WoodMason.ipynb to html
[NbConvertApp] Writing 352041 bytes to lab_06_WoodMason.html
